### Import package

In [1]:
import csv 
import requests 
import xml.etree.ElementTree as ET 
import re, collections
from lxml import etree
import os


### Read file

In [2]:
!dir C:\Users\laurent\Desktop\TestTableau
    

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est ECA6-2E6D

 R‚pertoire de C:\Users\laurent\Desktop\TestTableau

07/08/2019  10:30    <DIR>          .
07/08/2019  10:30    <DIR>          ..
07/08/2019  09:56             6ÿ919 IRIS.tds
07/08/2019  09:56             6ÿ919 tds_IRIS.tds
07/08/2019  10:24             9ÿ979 wb_IRIS.twb
               3 fichier(s)           23ÿ817 octets
               2 R‚p(s)  129ÿ605ÿ292ÿ032 octets libres


In [3]:
file_directory = 'C:\\Users\laurent\Desktop\TestTableau'
file_name = 'tds_IRIS.tds'#'Iris.twb'
#file_path=os.path.join(file_directory,file_name)
os.chdir(file_directory)


## Lxml xml Parsing

### Root 

In [4]:
tree = etree.parse(file_name)
root = tree.getroot()


### Beautiful tree

In [29]:
#os.environ["GIT_PYTHON_REFRESH"] = "quiet"
import git
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
sha

'6b2ef74705f9c5f01f7b4ebb30a71951329035fa'

In [61]:
os.listdir()

['.git',
 '.gitignore',
 'IRIS.tds',
 'tds_IRIS.tds',
 'wb_IRIS - Copie.twb',
 'wb_IRIS.twb',
 '~wb_IRIS__14952.twbr']

In [70]:
from collections import Counter
def PrettyTree(root,tree):
    nice_tree = collections.OrderedDict()
    cnt = Counter()
    print('', file=open("output.txt", "w"))
    for i,ElementNode in enumerate(root.iter()):
            #print(ElementNode,tree.getpath(ElementNode))
            path =re.sub('\[[0-9]+\]', '', tree.getpath(ElementNode))
            cnt[path] += 1
            if path not in nice_tree:
                nice_tree[path] = []
            nice_tree[path].extend(attrib for attrib in ElementNode.keys() if attrib not in nice_tree[path])
    for path, attribs in nice_tree.items():
        indent = int(path.count('/') - 1)
        print('{0}{1}: {2}({3}) [{4}]'.format('    ' * indent, indent, path.split('/')[-1], cnt[path],', '.join(attribs) if len(attribs) > 0 else '-'))
        print('{0}{1}: {2}({3}) [{4}]'.format('    ' * indent, indent, path.split('/')[-1], cnt[path],', '.join(attribs) if len(attribs) > 0 else '-'), file=open("output.txt", "a"))
    return True

#### Test beautiful Tree

https://python-forum.io/Thread-How-to-display-XML-tree-structure-with-Python

##### Alias pour filtre

In [105]:
root_wb.xpath("//*[@value]")
for node in root_wb.findall(".//*[@value]"):
    print(node.tag,node.keys(),node.get('value'))
    

preference ['name', 'value'] 24
preference ['name', 'value'] 26
semantic-value ['key', 'value'] "France"
column ['caption', 'datatype', 'name', 'param-domain-type', 'role', 'type', 'value'] 5
column ['caption', 'datatype', 'name', 'param-domain-type', 'role', 'type', 'value'] 5
aggregation ['value'] true
format ['attr', 'field', 'value'] Mon filtre espècenom hehe
breakdown ['value'] auto


##### Création et modification d'une colonne (champ calculé, paramètre, set,...)

In [101]:
root_wb.xpath("//*[@caption]")
for node in root_wb.findall(".//*[@caption]"):
    print(node.tag,node.keys(),node.get('caption'),node.get('datatype'))
    


named-connection ['caption', 'name'] IRIS None
column ['caption', 'datatype', 'name', 'role', 'type'] Add largeur longueur real
column ['caption', 'datatype', 'name', 'role', 'type'] largeur - longueur real
column ['caption', 'datatype', 'name', 'role', 'type'] Petal longueur real
column ['caption', 'datatype', 'name', 'role', 'type'] Petal largeur real
column ['caption', 'datatype', 'name', 'role', 'type'] Sepal longueur real
column ['caption', 'datatype', 'name', 'role', 'type'] Sepal largeur real
column ['caption', 'datatype', 'name', 'role', 'type'] Espèce numéro integer
column ['caption', 'datatype', 'name', 'role', 'type'] Espèce nom string
column ['caption', 'datatype', 'name', 'param-domain-type', 'role', 'type', 'value'] My param top N integer
column ['caption', 'datatype', 'name', 'param-domain-type', 'role', 'type', 'value'] My param top N integer
column ['caption', 'datatype', 'name', 'role', 'type'] Sepal largeur real
column ['caption', 'datatype', 'name', 'role', 'type'] 

##### Alias pour les champs textuels (changer data intérieure)

 <alias key='&quot; Setosa&quot;' value=' MonAliasSetosa'
        
 Problème ed création d'alias (en cas d'oublies à certaines lignes)

In [109]:
#version 2
tree_wb = etree.parse('wb_IRIS.twb')
root_wb = tree_wb.getroot()
PrettyTree(root_wb,tree_wb)

0: workbook(1) [original-version, source-build, source-platform, version]
    1: preferences(1) [-]
        2: preference(2) [name, value]
    1: datasources(1) [-]
        2: datasource(2) [inline, name, version, hasconnection]
            3: connection(1) [class]
                4: named-connections(1) [-]
                    5: named-connection(1) [caption, name]
                        6: connection(1) [class, cleaning, compat, dataRefreshTime, filename, interpretationMode, password, server, validate]
                4: relation(1) [connection, name, table, type]
                    5: columns(1) [gridOrigin, header, outcome]
                        6: column(6) [datatype, name, ordinal]
                4: metadata-records(1) [-]
                    5: metadata-record(7) [class]
                        6: remote-name(7) [-]
                        6: remote-type(7) [-]
                        6: local-name(6) [-]
                        6: parent-name(7) [-]
                       

True

In [240]:
tree_tds = etree.parse('tds_IRIS.tds')
root_tds = tree_tds.getroot()
PrettyTree(root)

0: datasource(1) [formatted-name, inline, source-platform, version]
    1: connection(1) [class]
        2: named-connections(1) [-]
            3: named-connection(1) [caption, name]
                4: connection(1) [class, cleaning, compat, dataRefreshTime, filename, interpretationMode, password, server, validate]
        2: relation(1) [connection, name, table, type]
            3: columns(1) [gridOrigin, header, outcome]
                4: column(6) [datatype, name, ordinal]
        2: metadata-records(1) [-]
            3: metadata-record(7) [class]
                4: remote-name(7) [-]
                4: remote-type(7) [-]
                4: local-name(6) [-]
                4: parent-name(7) [-]
                4: remote-alias(7) [-]
                4: ordinal(6) [-]
                4: local-type(6) [-]
                4: aggregation(7) [-]
                4: contains-null(7) [-]
                4: attributes(7) [-]
                    5: attribute(10) [datatype, name]
         

True

In [225]:
############### Test ###################
nice_tree = collections.OrderedDict()
cnt = Counter()
for i,ElementNode in enumerate(root.iter()):
        #print(ElementNode,tree.getpath(ElementNode))
        path =re.sub('\[[0-9]+\]', '', tree.getpath(ElementNode))
        cnt[path] += 1
        if path not in nice_tree:
            nice_tree[path] = []
        nice_tree[path].extend(attrib for attrib in ElementNode.keys() if attrib not in nice_tree[path])

for path, attribs in nice_tree.items():
    indent = int(path.count('/') - 1)
    print('{0}{1}: {2}({3}) [{4}]'.format('    ' * indent, indent, path.split('/')[-1], cnt[path],', '.join(attribs) if len(attribs) > 0 else '-'))

0: datasource(1) [formatted-name, inline, source-platform, version]
    1: connection(1) [class]
        2: named-connections(1) [-]
            3: named-connection(1) [caption, name]
                4: connection(1) [class, cleaning, compat, dataRefreshTime, filename, interpretationMode, password, server, validate]
        2: relation(1) [connection, name, table, type]
            3: columns(1) [gridOrigin, header, outcome]
                4: column(6) [datatype, name, ordinal]
        2: metadata-records(1) [-]
            3: metadata-record(7) [class]
                4: remote-name(7) [-]
                4: remote-type(7) [-]
                4: local-name(6) [-]
                4: parent-name(7) [-]
                4: remote-alias(7) [-]
                4: ordinal(6) [-]
                4: local-type(6) [-]
                4: aggregation(7) [-]
                4: contains-null(7) [-]
                4: attributes(7) [-]
                    5: attribute(10) [datatype, name]
         

## ElementTree xml Parsing

In [28]:
!more  C:\\Users\laurent\Desktop\Iris.twb 

<?xml version='1.0' encoding='utf-8' ?>

<!-- build 10500.18.0305.1200                               -->
<workbook original-version='10.5' source-build='10.5.2 (10500.18.0305.1200)' source-platform='win' version='10.5' xmlns:user='http://www.tableausoftware.com/xml/user'>
  <preferences>
    <preference name='ui.encoding.shelf.height' value='24' />
    <preference name='ui.shelf.height' value='26' />
  </preferences>
  <datasources>
    <datasource caption='Sheet1 (IRIS)' inline='true' name='federated.158axzb1r5gieb1e0lpaa1goumaw' version='10.5'>
      <connection class='federated'>
        <named-connections>
          <named-connection caption='IRIS' name='excel-direct.1qkk2b50o89opa1exgl8t1z07ct8'>
            <connection class='excel-direct' cleaning='no' compat='no' dataRefreshTime='' filename='C:/Users/laurent/Downloads/IRIS.xlsx' interpretationMode='0' password='' server='' validate='no' />
          </named-connection>
        </named-connections>
        <relation connection='


### Root 

In [78]:
tree = ET.parse(file_path)
root = tree.getroot()
print('tag:',root.tag,'attrib',root.attrib,'text:',root.text)

tag: datasource attrib {'formatted-name': 'federated.158axzb1r5gieb1e0lpaa1goumaw', 'inline': 'true', 'source-platform': 'win', 'version': '10.5'} text 
  


In [92]:
all_descendants = list(root.iter())
[node.tag for node in all_descendants]

['datasource',
 'connection',
 'named-connections',
 'named-connection',
 'connection',
 'relation',
 'columns',
 'column',
 'column',
 'column',
 'column',
 'column',
 'column',
 'metadata-records',
 'metadata-record',
 'remote-name',
 'remote-type',
 'local-name',
 'parent-name',
 'remote-alias',
 'ordinal',
 'local-type',
 'aggregation',
 'contains-null',
 'attributes',
 'attribute',
 'metadata-record',
 'remote-name',
 'remote-type',
 'local-name',
 'parent-name',
 'remote-alias',
 'ordinal',
 'local-type',
 'aggregation',
 'precision',
 'contains-null',
 'attributes',
 'attribute',
 'metadata-record',
 'remote-name',
 'remote-type',
 'local-name',
 'parent-name',
 'remote-alias',
 'ordinal',
 'local-type',
 'aggregation',
 'precision',
 'contains-null',
 'attributes',
 'attribute',
 'metadata-record',
 'remote-name',
 'remote-type',
 'local-name',
 'parent-name',
 'remote-alias',
 'ordinal',
 'local-type',
 'aggregation',
 'precision',
 'contains-null',
 'attributes',
 'attribute'

In [80]:
for child in root:
    print(child.tag)

connection
aliases
column
column
column
column
column
column
column
layout
semantic-values
date-options


### Child Root 

In [75]:
for child in root:
    print(">tag:",child.tag,">attrib:", child.attrib)

>tag: connection >attrib: {'class': 'federated'}
>tag: aliases >attrib: {'enabled': 'yes'}
>tag: column >attrib: {'datatype': 'integer', 'name': '[Number of Records]', 'role': 'measure', 'type': 'quantitative'}
>tag: column >attrib: {'caption': 'Petal longueur', 'datatype': 'real', 'name': '[Petal_length]', 'role': 'measure', 'type': 'quantitative'}
>tag: column >attrib: {'caption': 'Petal largeur', 'datatype': 'real', 'name': '[Petal_width]', 'role': 'measure', 'type': 'quantitative'}
>tag: column >attrib: {'caption': 'Sepal longueur', 'datatype': 'real', 'name': '[Sepal_length]', 'role': 'measure', 'type': 'quantitative'}
>tag: column >attrib: {'caption': 'Sepal largeur', 'datatype': 'real', 'name': '[Sepal_width]', 'role': 'measure', 'type': 'quantitative'}
>tag: column >attrib: {'caption': 'Espèce numéro', 'datatype': 'integer', 'name': '[Species_No]', 'role': 'measure', 'type': 'quantitative'}
>tag: column >attrib: {'caption': 'Espèce nom', 'datatype': 'string', 'name': '[Species_

### Search by tag

In [76]:
for neighbor in root.iter('column'):
    if 'caption' in neighbor.attrib:
        print(neighbor,neighbor.attrib,neighbor.text)

<Element 'column' at 0x00000280A8F71908> {'caption': 'Petal longueur', 'datatype': 'real', 'name': '[Petal_length]', 'role': 'measure', 'type': 'quantitative'} None
<Element 'column' at 0x00000280A8F71958> {'caption': 'Petal largeur', 'datatype': 'real', 'name': '[Petal_width]', 'role': 'measure', 'type': 'quantitative'} None
<Element 'column' at 0x00000280A8F719F8> {'caption': 'Sepal longueur', 'datatype': 'real', 'name': '[Sepal_length]', 'role': 'measure', 'type': 'quantitative'} None
<Element 'column' at 0x00000280A8F71AE8> {'caption': 'Sepal largeur', 'datatype': 'real', 'name': '[Sepal_width]', 'role': 'measure', 'type': 'quantitative'} None
<Element 'column' at 0x00000280A8F71B38> {'caption': 'Espèce numéro', 'datatype': 'integer', 'name': '[Species_No]', 'role': 'measure', 'type': 'quantitative'} None
<Element 'column' at 0x00000280A8F71B88> {'caption': 'Espèce nom', 'datatype': 'string', 'name': '[Species_name]', 'role': 'dimension', 'type': 'nominal'} None


Element.findall() finds only elements with a tag which are direct children of the current element. Element.find() finds the first child with a particular tag, and Element.text accesses the element’s text content. Element.get() accesses the element’s attributes:

### Writing

Once created, an Element object may be manipulated by directly changing its fields (such as Element.text), adding and modifying attributes (Element.set() method), as well as adding new children (for example with Element.append()).

In [38]:
traduction.keys()

dict_keys(['Petal length', 'Petal width', 'Sepal length', 'Sepal width', 'Species No', 'Species name'])

In [48]:

mydict={}
for neighbor in root.iter('column'):
    if 'caption' in neighbor.attrib:
            mydict[neighbor.attrib['name']]=neighbor.attrib['caption']
            

In [49]:
mydict

{'[Petal_length]': 'Petal longueur',
 '[Petal_width]': 'Petal largeur',
 '[Sepal_length]': 'Sepal longueur',
 '[Sepal_width]': 'Sepal largeur',
 '[Species_No]': 'Espèce numéro',
 '[Species_name]': 'Espèce nom'}

In [47]:
traduction={'Petal length':'Petal longueur',
'Petal width':'Petal largeur',
'Sepal length':'Sepal longueur',
'Sepal width':'Sepal largeur',
'Species No':'Espèce numéro',
'Species name':'Espèce nom'}
for neighbor in root.iter('column'):
    if 'caption' in neighbor.attrib:
        if neighbor.attrib['caption'] in list(traduction.keys())  :
            print(traduction[neighbor.attrib['caption']])
            neighbor.set('caption',traduction[neighbor.attrib['caption']])

            
            
tree.write('output.twb')
        

In [44]:
!more  output.xml

<workbook xmlns:ns0="http://www.tableausoftware.com/xml/user" original-version="10.5" source-build="10.5.2 (10500.18.0305.1200)" source-platform="win" version="10.5">
  <preferences>
    <preference name="ui.encoding.shelf.height" value="24" />
    <preference name="ui.shelf.height" value="26" />
  </preferences>
  <datasources>
    <datasource caption="Sheet1 (IRIS)" inline="true" name="federated.158axzb1r5gieb1e0lpaa1goumaw" version="10.5">
      <connection class="federated">
        <named-connections>
          <named-connection caption="IRIS" name="excel-direct.1qkk2b50o89opa1exgl8t1z07ct8">
            <connection class="excel-direct" cleaning="no" compat="no" dataRefreshTime="" filename="C:/Users/laurent/Downloads/IRIS.xlsx" interpretationMode="0" password="" server="" validate="no" />
          </named-connection>
        </named-connections>
        <relation connection="excel-direct.1qkk2b50o89opa1exgl8t1z07ct8" name="Sheet1" table="[Sheet1$]" type="table">
          <column

In [46]:
! dir

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est ECA6-2E6D

 R‚pertoire de C:\Users\laurent\Documents

06/08/2019  17:47    <DIR>          .
06/08/2019  17:47    <DIR>          ..
26/07/2019  14:39    <DIR>          .ipynb_checkpoints
23/07/2019  16:31    <DIR>          BI
24/07/2019  18:01             2ÿ226 Dictionnaire des donn‚es - Raccourci.lnk
06/08/2019  15:05    <DIR>          Extracts
31/07/2019  18:45    <DIR>          ludo_aws_vpn
12/07/2019  12:02    <DIR>          Mon dossier Tableau
29/07/2019  13:30            11ÿ098 mysql_driver.ipynb
06/08/2019  17:46             9ÿ851 output.xml
29/07/2019  18:13            96ÿ601 Parse_JL.ipynb
06/08/2019  17:47            33ÿ025 Tableau_api.ipynb
02/08/2019  16:30    <DIR>          Tests
17/07/2019  15:40    <DIR>          Vanessa_Martin
               5 fichier(s)          152ÿ801 octets
               9 R‚p(s)  130ÿ686ÿ349ÿ312 octets libres


## Tableau

In [9]:
#!pip install tableau_tools

In [2]:
from tableaudocumentapi import Workbook
sourceWB = Workbook('C:\\Users\laurent\Documents\Vanessa_Martin\Analyse_Hotel\Hotel_test_api.twb')

In [7]:
sourceWB.worksheets

['DATA',
 "Délais d'anticipation",
 'KPI',
 'KPI Lib',
 'Map',
 'NB Nuit',
 'Prestation et service',
 'Résumé',
 'Sheet 14',
 'Sheet 15',
 'Sheet 16',
 'Sheet 17',
 'Sheet 18',
 'Sheet 19',
 'Sheet 20',
 'Sheet 21',
 'Sheet 22',
 'Sheet 23',
 'Sheet 24',
 'Sheet 25',
 'Sheet 26',
 'Taux de couverture',
 'Top Hotel',
 'Top Ville',
 'Type Hotel']

In [9]:
sourceWB.datasources[0]